In [8]:
from galileojp.k3s import K3SGateway
from dotenv import load_dotenv
import warnings
from influxdb_client.client.warnings import MissingPivotFunction

warnings.simplefilter("ignore", MissingPivotFunction)


load_dotenv()

True

In [2]:
%env

{'PATH': '/home/silv/projects/silv-io/mulambda-experiments/.venv/bin:/home/silv/.pyenv/shims:/home/silv/.pyenv/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/lib/jvm/default/bin:/usr/bin/site_perl:/usr/bin/vendor_perl:/usr/bin/core_perl:/usr/lib/rustup/bin:/home/silv/.local/share/JetBrains/Toolbox/scripts',
 'INVOCATION_ID': '87fde994fe6345b3bab86127be4b23af',
 'XAUTHORITY': '/run/user/1000/.mutter-Xwaylandauth.BRY371',
 'GDMSESSION': 'gnome',
 'XDG_DATA_DIRS': '/home/silv/.local/share/flatpak/exports/share:/var/lib/flatpak/exports/share:/usr/local/share/:/usr/share/',
 'MOTD_SHOWN': 'pam',
 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus',
 'LANG': 'en_US.UTF-8',
 'XDG_SESSION_TYPE': 'wayland',
 'XDG_ACTIVATION_TOKEN': '<hidden>',
 'PS1': '(.venv) ',
 'XDG_CURRENT_DESKTOP': 'GNOME',
 'JOURNAL_STREAM': '8:30177',
 'DISPLAY': ':0',
 'MAIL': '/var/spool/mail/silv',
 'WAYLAND_DISPLAY': 'wayland-0',
 'USERNAME': 'silv',
 'SESSION_MANAGER': 'local/io:@/tmp/.ICE-unix/2176,unix/io

In [3]:
gw = K3SGateway.from_env()

In [9]:
gw.experiments().sort_values(by='CREATED', ascending=False).head(10)

/home/silv/projects/silv-io/mulambda-experiments/.venv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/silv/projects/silv-io/mulambda-experiments/.venv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,EXP_ID,NAME,CREATOR,START,END,CREATED,STATUS,metadata
7,202307211217-2a73,local,silvio,1.689935e+09,1.689935e+09,1.689935e+09,FINISHED,"{""test"": ""test"", ""exp_id"": ""202307211217-2a73""}"
6,202307200951-6527,test,silvio,1.689839e+09,1.689839e+09,1.689839e+09,FINISHED,"{""test"": ""test"", ""exp_id"": ""202307200951-6527""}"
5,202307200940-2320,test,silvio,1.689839e+09,1.689839e+09,1.689839e+09,FINISHED,"{""test"": ""test"", ""exp_id"": ""202307200940-2320""}"
4,202307200934-fab5,test,silvio,1.689838e+09,1.689839e+09,1.689838e+09,FINISHED,"{""test"": ""test"", ""exp_id"": ""202307200934-fab5""}"
3,202307200920-3479,test,silvio,1.689838e+09,1.689838e+09,1.689838e+09,FINISHED,"{""test"": ""test"", ""exp_id"": ""202307200920-3479""}"
2,202307130214-0821,test,silvio,1.689207e+09,1.689207e+09,1.689207e+09,FINISHED,"{""test"": ""test"", ""exp_id"": ""202307130214-0821""}"
1,202307130207-f479,test,silvio,1.689207e+09,1.689207e+09,1.689207e+09,FINISHED,"{""test"": ""test"", ""exp_id"": ""202307130207-f479""}"
0,202307122326-59e7,test,silvio,1.689197e+09,1.689197e+09,1.689197e+09,FINISHED,"{""test"": ""test"", ""exp_id"": ""202307122326-59e7""}"


In [5]:
exp_id = '202307200951-6527'

In [10]:
gw.telemetry(exp_id).head(10)

,value,exp_id,metric,node,ts,subsystem
ts,,,,,,
2023-07-20 07:51:02.377235174,2.525253,202307200951-6527,cpu,m10-controller-zone-a-0,1689839462.3772352,NaN
2023-07-20 07:51:03.378091574,1.005025,202307200951-6527,cpu,m10-controller-zone-a-0,1689839463.3780916,NaN
2023-07-20 07:51:04.376661301,1.010101,202307200951-6527,cpu,m10-controller-zone-a-0,1689839464.3766613,NaN
2023-07-20 07:51:05.375780821,1.010101,202307200951-6527,cpu,m10-controller-zone-a-0,1689839465.3757808,NaN
2023-07-20 07:51:06.376323462,2.538071,202307200951-6527,cpu,m10-controller-zone-a-0,1689839466.3763235,NaN
2023-07-20 07:51:07.375845909,1.515152,202307200951-6527,cpu,m10-controller-zone-a-0,1689839467.375846,NaN
2023-07-20 07:51:08.377199650,1.530612,202307200951-6527,cpu,m10-controller-zone-a-0,1689839468.3771996,NaN
2023-07-20 07:51:09.377016306,2.010050,202307200951-6527,cpu,m10-controller-zone-a-0,1689839469.3770163,NaN
2023-07-20 07:51:10.375964642,3.000000,202307200951-6527,cpu,m10-controller-zone-a-0,1689839470.3759646,NaN


In [11]:
gw.events(exp_id).head(10)

,value,exp_id,name,ts
ts,,,,
2023-07-20 07:51:09.253330469,"{""type"": ""start""}",202307200951-6527,mulambda,1689839469.2533305
2023-07-20 07:51:09.297752142,"{""type"": ""end""}",202307200951-6527,mulambda,1689839469.2977521
2023-07-20 07:51:09.360459089,"{""client_id"": ""experiment-client"", ""endpoint"":...",202307200951-6527,mulambda,1689839469.360459
2023-07-20 07:51:09.423598528,"{""client_id"": ""experiment-client"", ""endpoint"":...",202307200951-6527,mulambda,1689839469.4235985
2023-07-20 07:51:09.489501953,"{""client_id"": ""experiment-client"", ""endpoint"":...",202307200951-6527,mulambda,1689839469.489502
2023-07-20 07:51:09.553482056,"{""client_id"": ""experiment-client"", ""endpoint"":...",202307200951-6527,mulambda,1689839469.553482
2023-07-20 07:51:09.625619411,"{""client_id"": ""experiment-client"", ""endpoint"":...",202307200951-6527,mulambda,1689839469.6256194
2023-07-20 07:51:09.702177286,"{""client_id"": ""experiment-client"", ""endpoint"":...",202307200951-6527,mulambda,1689839469.7021773
2023-07-20 07:51:09.772459984,"{""client_id"": ""experiment-client"", ""endpoint"":...",202307200951-6527,mulambda,1689839469.77246
